# Formatting Functions

In [8]:
import scipy.io as sio
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from skimage.feature import hog
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import os
import scipy.sparse as sp
import scipy.io as sio
import time
import cuml
from cuml.svm import SVC as cuSVC
import cudf
from sklearn.decomposition import PCA
import scipy.stats
import librosa
from cuml.neighbors import KNeighborsClassifier as cuKNN
import cupy as cp
import h5py
from sklearn.preprocessing import StandardScaler
import gc
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout, Activation, ReLU, Input, LeakyReLU
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, LearningRateScheduler
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.utils import class_weight
from scipy.stats import mode

def rolling_window(arr, window_len, step):

    num_windows = (len(arr) - window_len) // step + 1
    windows = np.zeros((num_windows, window_len), dtype=arr.dtype)

    for i in range(num_windows):
        start = i * step
        end = start + window_len
        windows[i] = arr[start:end]

    return windows

def rolling_window_electrodes(arr, window_len, step):

    num_windows = (arr.shape[1] - window_len) // step + 1
    windows = np.zeros((arr.shape[0], num_windows, window_len), dtype=arr.dtype)

    for i in range(num_windows):
      start = i * step
      end = start + window_len
      windows[:, i] = arr[:, start:end]  # Slice along columns

    return windows


# turn labels into one hot representation
def one_hot_encoder(labels, gestures):

        one_hot = np.zeros((len(labels), gestures))

        for index, value in enumerate(labels):
            label_encode = int(value-1)
            one_hot[index][label_encode] = 1

        return one_hot

class LocallyConnectedLayer(layers.Layer):
    def __init__(self, filters, kernel_size, **kwargs):
        super(LocallyConnectedLayer, self).__init__(**kwargs)
        self.filters = filters
        self.kernel_size = kernel_size

    def build(self, input_shape):
        # Initialize the weights for each spatial location
        self.kernel = self.add_weight(
            name='kernel',
            shape=(self.kernel_size[0], self.kernel_size[1], input_shape[-1], self.filters),
            initializer='glorot_uniform',
            trainable=True
        )
        super(LocallyConnectedLayer, self).build(input_shape)

    def call(self, inputs):
        # Perform the "local" convolution manually without weight sharing
        return tf.nn.conv2d(inputs, self.kernel, strides=[1, 1, 1, 1], padding='SAME')

def lr_schedule(epoch):
    initial_lr = 0.002  # Initial learning rate
    drop_factor = 0.5  # Factor by which to reduce the learning rate
    epochs_drop = 10  # Interval (in epochs) to reduce the learning rate
    return initial_lr * (drop_factor ** (epoch // epochs_drop))

def generate_signal_permutations(Ns):

    SIS = ""  # String to store signal indices
    NSIS = 1  # Initial length of SIS

    # Start with the first signal sequence (hexadecimal '1')
    SIS = format(1, 'X')  # SIS starts with '1' in hexadecimal

    i = 1
    j = i + 1

    # Apply the permutation logic as per the original algorithm
    while i != j:
        if j > Ns:
            j = 1  # Wrap around if j exceeds Ns
        elif format(i, 'X') + format(j, 'X') not in SIS and format(j, 'X') + format(i, 'X') not in SIS:  # If pair not in SIS
            # Append the j-th signal index to SIS (in hexadecimal)
            SIS += format(j, 'X')  # Add j to the string in hexadecimal
            NSIS += 1  # Increment NSIS
            i = j  # Update i to current j
            j = i + 1  # Update j to i + 1
        else:
            j += 1  # Move j to the next signal sequence
    # Return the final Signal Index String (SIS) as a list of integers and NSIS
    return [int(x, 16) for x in SIS], NSIS


# TO RUN

In [9]:
import numpy as np
from google.colab import drive

# ONLY DB1 USES THIS FILE
database = 'DB1'

# ADJUST ACCORDINLY
evaluation = 'HHT'

# ADJUST DEPNDING ON WHAT FEATURES
num_features = 3

# ADJUST DEPNDING ON Signal Image or not
signal_image = False

# ok, so E2 and E3 adjusted means to adjust them gesture back to 1-N gestures, otherwise it is say 18-40 but depending on what gesture set went first
data_dict = {
        'DB1': {'E1': 12, 'E2': 17, 'E3': 23, 'E1_adjusted': 0, 'E2_adjusted': 0, 'E3_adjusted': 0, 'fs': 100, 'electrodes': 10, 'subjects': 27, 'train': [1, 3, 4, 6, 7, 8, 9], 'test': [2, 5, 10], 'window length': 20, 'step': 1},
        'DB2': {'E1': 18, 'E2': 24, 'E3': 10, 'E1_adjusted': 0, 'E2_adjusted': -17, 'E3_adjusted': -40, 'fs': 100, 'electrodes': 12, 'subjects': 40, 'train': [1, 3, 4, 6], 'test': [2, 5], 'window length': 20, 'step': 1},
        'DB3': {'E1': 18, 'E2': 24, 'E3': 10, 'E1_adjusted': 0, 'E2_adjusted': -17, 'E3_adjusted': -40, 'fs': 2000, 'electrodes': 12, 'subjects': 11, 'train': [1, 3, 4, 6], 'test': [2, 5], 'window length': 400, 'step': 20},
        'DB5': {'E1': 13, 'E2': 18, 'E3': 24, 'E1_adjusted': 0, 'E2_adjusted': 0, 'E3_adjusted': 0,'fs': 200, 'electrodes': 16, 'subjects': 10, 'train': [1, 3, 4, 6], 'test': [2, 5], 'window length': 40, 'step': 2}
        }

# use global dictionary
num_subjects = data_dict[database]['subjects']
fs = data_dict[database]['fs']
num_electrodes = data_dict[database]['electrodes']

train_trials =  data_dict[database]['train']
test_trials = data_dict[database]['test']
M, step = data_dict[database]['window length'], data_dict[database]['step']
num_freq_bins = int((fs / 2) / (1 / (1/fs * M)))
freq_bins = np.linspace(0, fs/2, num_freq_bins)

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# javascript simulated button click to prevent connection going idle
%%javascript
function ClickConnect(){
    console.log("Clicked on connect button");
    document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect,60000)

<IPython.core.display.Javascript object>

# Training

In [11]:

for exercise in  ['E1', 'E2', 'E3']:

  label_dict = {f'S{num}': [] for num in range(1, num_subjects+1)}
  features_dict = {f'S{num}': [] for num in range(1, num_subjects+1)}
  num_gestures = data_dict[database][exercise]

  # for all subjects
  for i in range(1,(num_subjects+1)):

      # DB1
      file = sio.loadmat(f'/content/drive/My Drive/uni/{database}/Electrode Data/S{i}_A1_{exercise}.mat')

      label = file['restimulus'].flatten()

      # load trial data, so training and testing data can be split according to trials
      trials = np.int8(file['rerepetition']).flatten()

      emg_data = file['emg'].T

      windowed_trials = rolling_window(trials, M, step)
      trials_arr = [np.max(arr) for arr in windowed_trials]

      train_feature_path = f'/content/drive/My Drive/uni/{database}/Features_down/DATA2_{exercise}_S{i}_features.h5'
      with h5py.File(train_feature_path, 'r') as feature_file:

          mav = feature_file['MAV'][:]
          mavs = feature_file['MAVS'][:]
          wap = feature_file['WAP'][:]
          zcr = feature_file['ZC'][:]
          ar1 = feature_file['ar1'][:]
          ar2 = feature_file['ar2'][:]
          ar3 = feature_file['ar3'][:]
          ar4 = feature_file['ar4'][:]
          wl = feature_file['WL'][:]
          ssc = feature_file['SSC'][:]
          var = feature_file['VAR'][:]
          iemg = feature_file['IEMG'][:]
          rms = feature_file['RMS'][:]


      # load either HHT or STFT data
      if evaluation == 'HHT':
          hht_path = f'/content/drive/My Drive/uni/{database}/HHT/{exercise}_S{i}_hht.h5'
      elif evaluation == 'STFT':
          hht_path = f'/content/drive/My Drive/uni/{database}/STFT/{exercise}_S{i}_stft.h5'

      with h5py.File(hht_path, 'r') as hht_file:
          mean_freq = hht_file['mean freq'][:]
          skew_freq = hht_file['skew freq'][:]
          psr = hht_file['psr'][:]
          #test_imfs = hht_file['num imfs'][:]
          peak_freq = hht_file['peak freq'][:]
          mean_power = hht_file['mean power'][:]
          kurt_freq = hht_file['kurt freq'][:]
          var_freq = hht_file['var freq'][:]

      if evaluation == 'HHT':
          mean_power = np.squeeze(mean_power)
          mean_freq = np.squeeze(mean_freq)
          psr = np.squeeze(psr)
          mean_power = np.squeeze(mean_power)
          mean_freq = np.squeeze(mean_freq)
          psr = np.squeeze(psr)

      # FEATURE SET 1
      #feature_matrices= [mav, mavs, wap, wl, ar1, ar2, ar3, ar4, mean_freq, psr]

      # FEATURE SET 2
      #feature_matrices = [iemg, var, wap, wl, ssc, zcr]

      # FEATURE SET 3
      feature_matrices = [mean_power, wl, mav]

      feature_images = np.stack(feature_matrices, axis=-1).transpose(1, 0, 2)

      # if feature image is then create permuation of rows
      if signal_image == True:

        # generate permuatations using function
        Ns = feature_images.shape[1]
        SIS, NSIS = generate_signal_permutations(Ns)
        print(SIS, NSIS)

        # slice features to create Signal Image
        features = np.zeros((feature_images.shape[0], NSIS, feature_images.shape[2]))
        for segment, image in enumerate(feature_images):
          for index, val in enumerate(SIS):
              features[segment][index] = feature_images[segment, val-1, :]

        # remove last 2 rows (to create even representation)
        features = features[:, 1:-1, :]

      # if not, just use unchanged array
      else:
        features = feature_images

      # window labels - only to predict whether gesture or not
      label_arr = rolling_window(label, M, step)
      label_arr = np.array([np.max(arr) for arr in label_arr])

      # split train / test data according to trial number
      train_index = []
      test_index = []
      for index, val in enumerate(trials):
        if val in train_trials:
          train_index.append(index)
        elif val in test_trials:
          test_index.append(index)

      # slice to get training data
      train_features = features[train_index, :]
      train_label_one = label_arr[train_index]

      # slice to get test data
      test_features = features[test_index, :]
      test_label_one = label_arr[test_index]

      print(train_features.shape, test_features.shape)

      # save gesture before bringing index back to 1, as it will only be compared - save these for reloading for validation
      # this is used, since Colab kept on crashing
      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_label_history_S{i}.h5', 'w') as f:
        f.create_dataset(f'{exercise}_S{i}', data=test_label_one, compression='gzip', compression_opts=8)

      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_features_history_S{i}.h5', 'w') as f:
          f.create_dataset(f'{exercise}_S{i}', data=test_features, compression='gzip', compression_opts=8)

      # adjust gestures back to 1-N - it's a list, since some datasets require adjusting
      train_label_ = [x + data_dict[database][f'{exercise}_adjusted'] for x in train_label_one]
      test_label_ = [x + data_dict[database][f'{exercise}_adjusted'] for x in test_label_one]

      print(np.unique(train_label_))
      print(np.unique(test_label_))

      # one encode labels
      train_label = one_hot_encoder(train_label_, num_gestures)
      test_label = one_hot_encoder(test_label_, num_gestures)

      print("fitting model now")

      # Define the model
      model = Sequential([

              Input(shape=(train_features.shape[1], num_features, 1)),
              BatchNormalization(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              Dropout(0.5),

              Flatten(),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),

              # predict what gesture out of that exercise - even if subject didn't perform that gesture (for unknown reason) it could be predicted
              Dense(num_gestures, activation='softmax'),


      ])

      model.compile(optimizer=Adam(learning_rate=0.002), loss='categorical_crossentropy', metrics=['accuracy'])
      early_stopping = EarlyStopping(monitor='val_accuracy', patience=15)

      checkpoint_weights = ModelCheckpoint(f'/content/drive/MyDrive/uni/{database}/Weights_unified/{exercise}_model_weights_S{i}.weights.h5',
                                      save_best_only=True,
                                      save_weights_only=True,
                                      monitor='val_accuracy',)
                                      #verbose=1)

      epoch_num = 200
      history = model.fit(
                train_features,
                train_label,
                epochs=epoch_num,
                batch_size=32,
                validation_data=(test_features, test_label),
                callbacks=[checkpoint_weights, LearningRateScheduler(lr_schedule), early_stopping],
                )


(26052, 10, 3) (11648, 10, 3)
[ 1  2  3  4  5  6  7  8  9 10 11 12]
[ 1  2  3  4  5  6  7  8  9 10 11 12]
fitting model now
Epoch 1/200
815/815 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - accuracy: 0.5549 - loss: 1.3902 - val_accuracy: 0.8085 - val_loss: 0.5415 - learning_rate: 0.0020
Epoch 2/200
165/815 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.8222 - loss: 0.4989

KeyboardInterrupt: 

# Validation

In [ ]:
from sklearn.metrics import accuracy_score
import seaborn as sns
import scipy.sparse as sp
import matplotlib.pyplot as plt

tot_num_gestures = data_dict[database]['E1'] + data_dict[database]['E2'] + data_dict[database]['E3']

combined_cm = np.zeros((tot_num_gestures, tot_num_gestures), dtype=int)
accuracy_dict = {'E1': [], 'E2': [], 'E3': []}

# redo just in case it timed out
Ns = data_dict[database]['electrodes']
SIS, NSIS = generate_signal_permutations(Ns)

for exercise in ['E1', 'E2', 'E3']:

  num_gestures = data_dict[database][exercise]

  # Define the model
  model = Sequential([

              # CHANGE INPUT SHAPE AS APPROPRIATE
              Input(shape=(NSIS-2, num_features, 1)),
              BatchNormalization(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              Conv2D(64, (3,3), padding='same', strides=(1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              LocallyConnectedLayer(64, (1,1)),
              BatchNormalization(),
              ReLU(),

              Dropout(0.5),

              Flatten(),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),
              Dense(512),
              BatchNormalization(),
              ReLU(),
              Dropout(0.65),

              # predict what gesture out of that exercise - even if subject didn't perform that gesture (for unknown reason) it could be predicted
              Dense(num_gestures, activation='softmax'),
  ])

  for i in range(1, num_subjects+1):

      # reload train / test features
      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_features_history_S{i}.h5', 'r') as f:
        test_features = f[f'{exercise}_S{i}'][:]

      with h5py.File(f'/content/drive/MyDrive/uni/{database}/History_unified/{exercise}_label_history_S{i}.h5', 'r') as f:
        test_label = f[f'{exercise}_S{i}'][:]


      # load labels and features for subject
      model.load_weights(f'/content/drive/MyDrive/uni/{database}/Weights_unified/{exercise}_model_weights_S{i}.weights.h5')

      print(np.unique(test_label))
      # compile the model
      predictions = model.predict(test_features)
      predicted_classes = np.argmax(predictions, axis=1)

      accuracy = accuracy_score(test_label-1, predicted_classes)
      print(f'{exercise} S{i} Accuracy: {accuracy*100:.2f}%')

      # create confusion matrix
      for pred, actual in zip(predicted_classes, test_label):
        #adjust as it is zero indexed and there''s no at-rest gesture
        if exercise == 'E2':
          pred += 11
          actual += 11
        elif exercise == 'E3':
          pred += 28
          actual += 28
        combined_cm[int(actual), int(pred)] += 1

      accuracy_dict[exercise].append(accuracy*100)

for key in accuracy_dict.keys():
  print(f'Average accuracy for {key}: {np.mean(accuracy_dict[key])}, and std: {np.std(accuracy_dict[key], ddof=1)}')
  print(f'num elements {len(accuracy_dict[key])}')

ye = []
for val in accuracy_dict.values():
  ye.extend(val)
print(f'Overall average: {np.mean(ye)} and std: {np.std(ye, ddof=1)}')

cm_path = f'/content/drive/MyDrive/uni/{database}/Confusion_matrices/{database}_confusion_matrix_stft.npz'
sp.save_npz(cm_path, sp.csr_matrix(combined_cm))

# adjust labels, because zero is not included and all exercises - gesture 1 indexed from 0 as there is not at-rest gesture!!
display_labels = [str(i) for i in range(1, tot_num_gestures+1)]

plt.figure(figsize=(8, 6))
sns.heatmap(combined_cm, annot=False, cmap='inferno', fmt='g', cbar=True, square=True)
plt.xticks(ticks=np.arange(0, len(display_labels), 2), labels=[display_labels[i] for i in range(0, len(display_labels), 2)], rotation=45)
plt.yticks(ticks=np.arange(0, len(display_labels), 2), labels=[display_labels[i] for i in range(0, len(display_labels), 2)], rotation=45)

plt.xlabel('Predicted Labels', fontsize=12)
plt.ylabel('True Labels', fontsize=12)
plt.title(f"Classification Accuracy of {database} for {data_dict[database]['subjects']} Subjects")
plt.show()